In [1]:
import numpy as np
from pandas import read_csv

In [2]:
from scipy.stats import norm

In [3]:
def getS0(name='', market='BSE', index=False):
    fields=['Close']
    if index:
        data = read_csv('./Data/'+market.lower()+'data1.csv', usecols=fields, index_col=False)
    else:
        data = read_csv('./Data/'+market+'Data/'+name+'.csv', usecols=fields, index_col=False)
    return data.iloc[-1]['Close']

In [12]:
def getHistoricalVolatility(data):
    close = np.array(data['Close'])
    change = np.zeros(len(close)-1)
    for i in range(1, len(close)):
        change[i-1] = (close[i]-close[i-1])/close[i-1]*100
    historicalVolatility = np.std(change)*(252**0.5)
    return historicalVolatility

In [5]:
def getCall(S, K, r, t, sig):
    d1 = (np.log(S/K)+t*(r+(sig**2)/2))/(sig*(t**0.5))
    d2 = d1-sig*(t**0.5)
    Nd1 = norm.cdf(d1)
    Nd2 = norm.cdf(d2)
#     print(d1, Nd1, d2, Nd2)
    C = S*Nd1 - K*np.exp(-r*t)*Nd2
    return C

In [6]:
def getPut(S, K, r, t, sig):
    d1 = (np.log(S/K)+t*(r+(sig**2)/2))/(sig*(t**0.5))
    d2 = d1-sig*(t**0.5)
    Nd1 = norm.cdf(-d1)
    Nd2 = norm.cdf(-d2)
#     print(d1, Nd1, d2, Nd2)
    P = K*np.exp(-r*t)*Nd2 - S*Nd1
    return P

In [7]:
stock_name=['ABB', 'AXISBANK', 'BALMLAWRIE', 'BHARTIARTL', 
          'CUMMINSIND', 'EMAMILTD', 'GODREJIND', 
          'HDFCBANK', 'HEROMOTOCO', 'HINDUNILVR', 
          'INFY', 'IOC', 'ITC', 'LUPIN', 'M&M', 
          'MAHABANK', 'NTPC', 'SBIN', 'SHREECEM', 'TCS']

In [ ]:
fields=['Date', 'Open', 'Close']
if index:
    data = read_csv('./Data/'+market.lower()+'data1.csv', usecols=fields, index_col=False)
else:
    data = read_csv('./Data/'+market+'Data/'+name+'.csv', usecols=fields, index_col=False)

In [ ]:
data = data.iloc[1213:]

In [8]:
name = 'INFY'
S0 = data.iloc[-1]['Close']
A = np.arange(0.5, 1.6, 0.1)
K = S0
r = 0.05
t = 126/252
sig = getHistoricalVolatility(data)

callPrice = np.zeros(len(A))
putPrice = np.zeros(len(A))

In [9]:
for i, a in enumerate(A):
    callPrice[i] = getCall(S0, a*K, r, t, sig)
    putPrice[i] = getPut(S0, a*K, r, t, sig)

In [10]:
callPrice

array([659.85, 659.85, 659.85, 659.85, 659.85, 659.85, 659.85, 659.85,
       659.85, 659.85, 659.85])

In [11]:
putPrice

array([321.77912273, 386.13494727, 450.49077182, 514.84659636,
       579.20242091, 643.55824545, 707.91407   , 772.26989454,
       836.62571909, 900.98154363, 965.33736818])